<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#特征选择" data-toc-modified-id="特征选择-1">特征选择</a></span></li><li><span><a href="#数据获取" data-toc-modified-id="数据获取-2">数据获取</a></span></li><li><span><a href="#特征处理" data-toc-modified-id="特征处理-3">特征处理</a></span></li><li><span><a href="#标签处理" data-toc-modified-id="标签处理-4">标签处理</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-5">模型训练</a></span><ul class="toc-item"><li><span><a href="#独占城市模型训练" data-toc-modified-id="独占城市模型训练-5.1">独占城市模型训练</a></span></li><li><span><a href="#竞争城市模型训练" data-toc-modified-id="竞争城市模型训练-5.2">竞争城市模型训练</a></span></li><li><span><a href="#交叉lr模型（单机未用）" data-toc-modified-id="交叉lr模型（单机未用）-5.3">交叉lr模型（单机未用）</a></span></li></ul></li><li><span><a href="#模型评估" data-toc-modified-id="模型评估-6">模型评估</a></span></li><li><span><a href="#参考" data-toc-modified-id="参考-7">参考</a></span></li></ul></div>

In [4]:
import os
import re
import time
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
warnings.filterwarnings('ignore')

/home/ai.liuxuanheng/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/ai.liuxuanheng/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)



### 特征选择
- 买卡后影响的需过滤的特征：订单、卡、收入相关特征
- 同时影响是否买卡和gmv的特征：
    - dws.dws_pt_user_picture_mild_summary_da： 年龄、性别、注册年份、注册月份、注册日份、注册周几、is_certified（是否实名）、province_id、city_id
    - dim.dim_user_info_df： bank_card（是否绑银行卡）、client_type（登陆应用类型）、是否输身份证号
    - ads.ads_it_user_picture_da： phone（是否绑手机号）、openid（是否绑微信号）、sys_ver、brand（手机系统、品牌）、active_count_180d（近180天打开应用次数）、service_type（1 免密支付 2 自动续费 null）
    - 城市信息：竞争情况、车辆、车效

### 数据获取

In [ ]:
df = get_df_from_csv_table(query_presto(f"""
with t1 as (
select 
 gender -- 性别,0未知,1男,2女 
,age -- 年龄 
,user_id -- 用户id 
,city_id -- 用户所在城市ID 
,year(register_time) y -- 注册时间 年
,month(register_time) m -- 注册时间 月
,day(register_time) d -- 注册时间 日
,case mod( date_diff('day',cast('2010-02-01' as date), register_time),7)
    when 0 then 1
    when 1 then 2
    when 2 then 3
    when 3 then 4
    when 4 then 5
    when 5 then 6
    when 6 then 7
  end AS day_of_week
,is_certified -- 是否实名认证 
,province_id -- 用户所在省份id 
,event_day -- 日期
from dws.dws_pt_user_picture_mild_summary_da -- 用户画像轻度汇总表表
where 1=1 
and event_day BETWEEN '{start}' and '{end}'  
and order_cnt_d>0
-- order by rand()
-- limit 50000
),
t2 as (
 select 
 user_id uid2-- 用户id 
,if(idcard='unknow',0,1) is_idcard -- 是否注册身份证号 
,if(bank_card='unknow',0,1) is_bank_card -- 银行卡卡号 
,client_type -- 用户登录类型 Android,ios,Web,Web-1:邀请好友活动,Web-2运营活动,Web-3女神活动,WxMP:微信小程序 
,event_day e2-- 操作日期 
from dim.dim_user_info_df -- 用户信息维度表
where 1=1 
and event_day BETWEEN '{start}' and '{end}'   
-- limit 100
),
t3 as (
 select 
 user_id uid3-- 用户id 
,mobile -- 手机号 
,if(openid is null,0,1) is_openid -- 是否绑定微信号 
,sys_ver -- 手机系统 
,brand -- 品牌 
,active_count_180d -- 近180日打开应用次数 
,service_type -- 微信支付授权类型：1 免密支付  2 自动续费 
,op_mode -- 运营模式：1 直营，2 代理 
,is_open_wepay_score -- 是否开通过微信支付分 1开通过 0 未开通过 
,event_day e3-- 日期
from ads.ads_it_user_picture_da --  用户画像标签
where 1=1 
and event_day BETWEEN '{start}' and '{end}'   
-- limit 100
 ),
 t4 as (
 select 
 city_id city_id2-- 城市id 
,city_comp_type -- 城市竞争类型 
,real_input_bike_cnt_d -- 当日实际投放车辆数 
,useable_bike_cnt_d -- 当日可用车辆数 
,used_bike_cnt_d -- 当日使用车辆数 
,bike_valid_d -- 当日车效 
,move_bike_valid_d -- 当日动车效 
,event_day e4-- yyyyMMdd 
from ads.ads_bi_ci_bike_supply_analysis_da -- 城市经营-供给分析，city_id粒度
where 1=1 
and event_day BETWEEN '{start}' and '{end}'   
-- limit 100
 )

select *

from t1 
    join t2 on t1.user_id = t2.uid2 and t1.event_day=t2.e2
    join t3 on t1.user_id = t3.uid3 and t1.event_day=t3.e3
    join t4 on t1.city_id = t4.city_id2 and t1.event_day=t4.e4

"""))

In [3]:
user_df = pd.read_csv('user_matching_feature.csv')
print(user_df.shape)
user_df.head()

(7470112, 48)


,Unnamed: 0,gender,age,user_id,city_id,y,m,d,day_of_week,is_certified,...,is_open_wepay_score,e3,city_id2,city_comp_type,real_input_bike_cnt_d,useable_bike_cnt_d,used_bike_cnt_d,bike_valid_d,move_bike_valid_d,e4
0,0,2,49,2105688,98,2018,7,21,6,1,...,1,20230106,98,两者都有,1400.0,1395.0,737.0,1.7857,3.3921,20230106
1,1,2,34,2283602,98,2018,8,6,1,1,...,1,20230106,98,两者都有,1400.0,1395.0,737.0,1.7857,3.3921,20230106
2,2,2,22,2852363,98,2018,9,23,7,1,...,1,20230106,98,两者都有,1400.0,1395.0,737.0,1.7857,3.3921,20230106
3,3,1,48,54167274,98,2021,6,5,6,1,...,0,20230106,98,两者都有,1400.0,1395.0,737.0,1.7857,3.3921,20230106
4,4,2,48,56526458,98,2021,6,28,1,1,...,0,20230106,98,两者都有,1400.0,1395.0,737.0,1.7857,3.3921,20230106


In [4]:
user_df.columns

Index(['Unnamed: 0', 'gender', 'age', 'user_id', 'city_id', 'y', 'm', 'd',
       'day_of_week', 'is_certified', 'province_id', 'event_day', 'uid4',
       'user_flag', 'flag_start_date', 'flag_end_date', 'arpu_start_date',
       'arpu_end_date', 'gmv', 'buy_ride_card_amt',
       'ride_order_should_pay_amt', 'buy_ride_card_cnt', 'order_cnt',
       'coupon_cnt', 'e5', 'uid2', 'is_idcard', 'is_bank_card', 'client_type',
       'e2', 'uid3', 'mobile', 'is_openid', 'sys_ver', 'brand',
       'active_count_180d', 'service_type', 'op_mode', 'is_open_wepay_score',
       'e3', 'city_id2', 'city_comp_type', 'real_input_bike_cnt_d',
       'useable_bike_cnt_d', 'used_bike_cnt_d', 'bike_valid_d',
       'move_bike_valid_d', 'e4'],
      dtype='object')

### 特征处理

In [5]:
def get_sys(x):
    try:
        return 'ios' if re.search(r'iOS', x, re.IGNORECASE) else 'android'
    except:
        return 'android'
user_df['re_sys_ver'] = user_df.sys_ver.map(lambda x:get_sys(x))

user_df['re_active_count_180d']=pd.qcut(user_df.active_count_180d,30)

values = {'real_input_bike_cnt_d': -1, 'useable_bike_cnt_d': -1, 'used_bike_cnt_d': -1, 'bike_valid_d': -1, 'move_bike_valid_d': -1}
user_df.fillna(value=values,inplace=True) 

user_df['re_real_input_bike_cnt_d']=pd.qcut(user_df.real_input_bike_cnt_d,10)
user_df['re_useable_bike_cnt_d']=pd.qcut(user_df.useable_bike_cnt_d,10)
user_df['re_used_bike_cnt_d']=pd.qcut(user_df.used_bike_cnt_d,10)
user_df['re_bike_valid_d']=pd.qcut(user_df.bike_valid_d,10)
user_df['re_move_bike_valid_d']=pd.qcut(user_df.move_bike_valid_d,10)

user_df['re_city_comp_type'] = user_df.city_comp_type.map(lambda x:1 if re.search('松果',x) else 0)


category_needX_cols = ['age','city_id','y','province_id','client_type',\
                       're_sys_ver','re_active_count_180d','service_type',\
                      'op_mode','re_real_input_bike_cnt_d','re_useable_bike_cnt_d',\
                       're_used_bike_cnt_d','re_bike_valid_d','re_move_bike_valid_d']
category_No_needX_cols = ['gender','m','d','day_of_week','is_certified',\
                         'is_idcard','is_bank_card','is_openid','is_open_wepay_score',\
                        're_city_comp_type']

user_df[category_needX_cols+category_No_needX_cols]

from sklearn.preprocessing import LabelEncoder

for feature in category_needX_cols:
    lbe = LabelEncoder()
    user_df[feature] = lbe.fit_transform(user_df[feature])

### 标签处理
- 持卡核销>0为正例，持卡核销>0.5为正例？
- 区分竞争和独占城市训练模型

In [6]:
user_df['use_card_ride_ratio'] = user_df.coupon_cnt/user_df.order_cnt
user_df['label'] = user_df.use_card_ride_ratio.map(lambda x:1 if x>0. else 0)

In [7]:
user_df.label.value_counts()

0    5927390
1    1542722
Name: label, dtype: int64

In [8]:
user_df.re_city_comp_type.value_counts()

0    4361335
1    3108777
Name: re_city_comp_type, dtype: int64

In [9]:
# 独占城市样本
single_df = user_df[user_df.re_city_comp_type==1]
single_df.label.value_counts()

0    2498257
1     610520
Name: label, dtype: int64

In [10]:
# 竞争城市样本
battle_df = user_df[user_df.re_city_comp_type==0]
battle_df.label.value_counts()

0    3429133
1     932202
Name: label, dtype: int64

### 模型训练
- causalml使用带交叉验证的lr，单机训练过慢，采用普通lr

In [11]:
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression

#### 独占城市模型训练

In [18]:
# 独占
kwargs = {
    "penalty": "elasticnet",
    "solver": "saga",
    "C": 1.,
    "l1_ratio": 1e-3,
    "random_state": 42,
}
lr = LogisticRegression(**kwargs)

In [19]:
single_train_X = single_df[category_needX_cols+category_No_needX_cols]
single_train_Y = single_df['label']

In [20]:
lr.fit(single_train_X,single_train_Y)

LogisticRegression(l1_ratio=0.001, penalty='elasticnet', random_state=42,
                   solver='saga')

- 概率预测，即倾向分

In [22]:
lr.predict_proba(single_train_X)

array([[0.68360859, 0.31639141],
       [0.66121132, 0.33878868],
       [0.77676088, 0.22323912],
       ...,
       [0.77349984, 0.22650016],
       [0.80130999, 0.19869001],
       [0.75082972, 0.24917028]])

In [28]:
single_df['p_score'] = lr.predict_proba(single_train_X)[:,1]

In [29]:
single_df.head()

,Unnamed: 0,gender,age,user_id,city_id,y,m,d,day_of_week,is_certified,...,re_active_count_180d,re_real_input_bike_cnt_d,re_useable_bike_cnt_d,re_used_bike_cnt_d,re_bike_valid_d,re_move_bike_valid_d,re_city_comp_type,use_card_ride_ratio,label,p_score
5984,5984,1,25,26595960,217,3,6,3,3,1,...,14,2,2,1,0,1,1,0.0,0,0.316391
5985,5985,2,10,24144842,217,3,5,4,1,1,...,0,2,2,1,0,1,1,0.0,0,0.338789
5986,5986,2,16,24334289,217,3,5,6,3,1,...,19,2,2,1,0,1,1,0.0,0,0.223239
5987,5987,1,27,79090240,217,5,7,22,5,1,...,21,2,2,1,0,1,1,0.0,0,0.287421
5988,5988,2,18,26249232,217,3,5,30,6,1,...,11,2,2,1,0,1,1,0.0,0,0.285087


In [30]:
single_df['p_score'] = single_df['p_score'].astype('float')

In [31]:
pd.qcut(single_df['p_score'],10)

5984       (0.264, 0.327]
5985       (0.327, 0.758]
5986       (0.197, 0.225]
5987       (0.264, 0.327]
5988       (0.264, 0.327]
                ...      
7468677    (0.327, 0.758]
7468678    (0.264, 0.327]
7470109    (0.225, 0.264]
7470110    (0.197, 0.225]
7470111    (0.225, 0.264]
Name: p_score, Length: 3108777, dtype: category
Categories (10, interval[float64]): [(0.0324, 0.0968] < (0.0968, 0.117] < (0.117, 0.136] < (0.136, 0.155] ... (0.197, 0.225] < (0.225, 0.264] < (0.264, 0.327] < (0.327, 0.758]]

In [ ]:
# 写入hive
spark_df.registerTempTable('tmptb')

spark.sql("""insert overwrite table ai.dws_smart_sales_card_user_propensity_da partition(event_day) 
select 
        user_id，age,city_id,y,province_id,client_type,
        re_sys_ver,re_active_count_180d,service_type,
        op_mode,re_real_input_bike_cnt_d,re_useable_bike_cnt_d,
        re_used_bike_cnt_d,re_bike_valid_d,re_move_bike_valid_d,
        gender,m,d,day_of_week,is_certified,
        is_idcard,is_bank_card,is_openid,is_open_wepay_score,
        re_city_comp_type,
        p_score,event_day
from 
        tmptb
""")



#### 竞争城市模型训练

In [43]:
# 竞争
lr2 = LogisticRegression(**kwargs)

In [44]:
battle_train_X = battle_df[category_needX_cols+category_No_needX_cols]
battle_train_Y = battle_df['label']

In [45]:
lr2.fit(battle_train_X,battle_train_Y)

LogisticRegression(l1_ratio=0.001, penalty='elasticnet', random_state=42,
                   solver='saga')

In [47]:
battle_df['p_score'] = lr2.predict_proba(battle_train_X)[:,1]

In [48]:
battle_df['p_score'] = battle_df['p_score'].astype('float')
pd.qcut(battle_df['p_score'],10)

0          (0.186, 0.202]
1          (0.186, 0.202]
2          (0.186, 0.202]
3          (0.377, 0.706]
4          (0.377, 0.706]
                ...      
7470104    (0.157, 0.171]
7470105    (0.186, 0.202]
7470106    (0.171, 0.186]
7470107    (0.123, 0.142]
7470108     (0.26, 0.377]
Name: p_score, Length: 4361335, dtype: category
Categories (10, interval[float64]): [(0.0499, 0.123] < (0.123, 0.142] < (0.142, 0.157] < (0.157, 0.171] ... (0.202, 0.223] < (0.223, 0.26] < (0.26, 0.377] < (0.377, 0.706]]

In [49]:
res2 = battle_df[['user_id']+category_needX_cols+category_No_needX_cols+['p_score','event_day']]
print(res2.shape)
res2.to_csv('竞争城市用户特征及倾向得分.csv')

(4361335, 27)


#### 交叉lr模型（单机未用）

In [ ]:
# 交叉lr 
train_X = user_df[category_needX_cols+category_No_needX_cols]
train_Y = user_df['label']

p_model = ElasticNetPropensityModel()
p = p_model.fit_predict(train_X, train_Y)

In [ ]:
# 倾向分分布
user_df['p_score'] = p
pd.qcut(user_df['p_score'],10)

### 模型评估
- 理论依据：倾向得分相同时，X的分布应该趋近一致
- 方法：standard mean difference (SMD)

$$
SMD=\frac{\left(\bar X_{1M}-\bar X_{0M}\right)}{\sqrt{0.5\cdot\left(V_{1M}(X)+V_{0M}(X)\right)}}
$$

In [32]:
# 匹配前
create_table_one(data=single_df,
                 treatment_col='label',
                 features=category_needX_cols+category_No_needX_cols)

,Control,Treatment,SMD
Variable,,,
n,2498257,610520,
age,15.58 (11.07),16.86 (11.11),0.1152
city_id,261.84 (189.14),249.03 (197.92),-0.0662
client_type,6.52 (1.73),6.50 (1.76),-0.0101
d,15.63 (9.12),15.86 (9.13),0.0245
day_of_week,4.18 (2.03),4.12 (2.03),-0.0291
gender,1.43 (0.49),1.39 (0.49),-0.0712
is_bank_card,0.00 (0.01),0.00 (0.01),0.003
is_certified,1.00 (0.02),1.00 (0.02),0.001


In [50]:
# 匹配后
psm = NearestNeighborMatch(replace=False,
                           ratio=1,
                           random_state=42)

user_df_matched = psm.match_by_group(data=single_df,
                             treatment_col='label',
                             score_cols=['p_score'],
                             groupby_col='re_active_count_180d')

create_table_one(data=user_df_matched,
                 treatment_col='label',
                 features=category_needX_cols+category_No_needX_cols)

,Control,Treatment,SMD
Variable,,,
n,600722,600722,
age,16.51 (11.54),16.78 (11.07),0.0238
city_id,248.38 (183.10),249.29 (198.28),0.0048
client_type,6.50 (1.78),6.50 (1.76),-0.0003
d,15.88 (9.11),15.84 (9.13),-0.0036
day_of_week,4.12 (2.03),4.12 (2.02),0.0016
gender,1.40 (0.49),1.39 (0.49),-0.0071
is_bank_card,0.00 (0.01),0.00 (0.01),0.0042
is_certified,1.00 (0.03),1.00 (0.02),0.0083


### 分层匹配

In [ ]:
with matching_detail as (
    select t1.user_id as treatment_userid,
      t1.score as treatment_pscore,
      t2.user_id as control_userid,
      t2.score as control_pscore,
      row_number() over (partition by t1.user_id order by abs(t1.score-t2.score) asc) as rn
    from propensity_score_treatment t1
    left join propensity_score_control t2
      -- 分层匹配
        on t1.gender = t2.gender and round(t1.score, 1)*10 =  round(t2.score, 1)*10
    where abs(t1.score-t2.score) <= 0.05 -- caliper matching
)
select * from matching_detail where rn = 1  # rn大于1时为多邻居/radius匹配

In [ ]:
#*** version='full-user-info' 用户基础数据分区，避免多表大数据量关联
#*** 层次、竞争类型、历史骑行单量、历史gmv等字段匹配

"""
with user_t as (

select 
user_id_t -- 干预组用户id 
,user_flag_t -- 干预组用户层次 
,city_comp_type_t -- 干预组用户所在城市竞争类型,独占1，竞争0 
,event_day_t -- 干预组用户开始观测日期 
,score_t -- 干预组用户倾向分 
,buy_ride_card_amt_t -- 干预组用户观测期间购卡gmv 
,ride_order_should_pay_amt_t -- 干预组用户观测期间骑行应付gmv 
,gmv_t -- 干预组用户观测期间总gmv 
,buy_ride_card_cnt_t -- 干预组用户观测期间买卡数量 
,use_card_ride_ratio_t -- 干预组用户观测期间用骑行卡订单占比 
,order_cnt_t -- 干预组用户观测期间订单数量 
,coupon_cnt_t -- 干预组用户观测期间核销订单数量 
,order_cnt_15d_t -- 干预组用户观测开始前15天订单 
,order_cnt_30d_t -- 干预组用户观测开始前30天订单 
,order_cnt_60d_t -- 干预组用户观测开始前60天订单 
,order_cnt_90d_t -- 干预组用户观测开始前90天订单 
,gmv_amt_15d_t -- 干预组用户观测开始前15天gmv 
,gmv_amt_30d_t -- 干预组用户观测开始前30天gmv 
,gmv_amt_90d_t -- 干预组用户观测开始前90天gmv 
,is_has_ride_card_t -- 干预组用户观测日期当天是否有卡 
from ai.dws_smart_sales_card_user_matching_detail_da -- undefined  
where 1=1 
and version='full-user-info'
and use_card_ride_ratio_t>0
),
 user_c as (

select 
user_id_t -- 干预组用户id 
,user_flag_t -- 干预组用户层次 
,city_comp_type_t -- 干预组用户所在城市竞争类型,独占1，竞争0 
,event_day_t -- 干预组用户开始观测日期 
,score_t -- 干预组用户倾向分 
,buy_ride_card_amt_t -- 干预组用户观测期间购卡gmv 
,ride_order_should_pay_amt_t -- 干预组用户观测期间骑行应付gmv 
,gmv_t -- 干预组用户观测期间总gmv 
,buy_ride_card_cnt_t -- 干预组用户观测期间买卡数量 
,use_card_ride_ratio_t -- 干预组用户观测期间用骑行卡订单占比 
,order_cnt_t -- 干预组用户观测期间订单数量 
,coupon_cnt_t -- 干预组用户观测期间核销订单数量 
,order_cnt_15d_t -- 干预组用户观测开始前15天订单 
,order_cnt_30d_t -- 干预组用户观测开始前30天订单 
,order_cnt_60d_t -- 干预组用户观测开始前60天订单 
,order_cnt_90d_t -- 干预组用户观测开始前90天订单 
,gmv_amt_15d_t -- 干预组用户观测开始前15天gmv 
,gmv_amt_30d_t -- 干预组用户观测开始前30天gmv 
,gmv_amt_90d_t -- 干预组用户观测开始前90天gmv 
,is_has_ride_card_t -- 干预组用户观测日期当天是否有卡 
from ai.dws_smart_sales_card_user_matching_detail_da -- undefined  
where 1=1 
and version='full-user-info'
and use_card_ride_ratio_t=0
)

insert overwrite table ai.dws_smart_sales_card_user_matching_detail_da partition(version='v5-no-churn-3d')
select 
    user_id_t -- 干预组用户id 
    ,user_flag_t -- 干预组用户层次 
    ,city_comp_type_t -- 干预组用户所在城市竞争类型,独占1，竞争0 
    ,event_day_t -- 干预组用户开始观测日期 
    ,score_t -- 干预组用户倾向分 
    ,buy_ride_card_amt_t -- 干预组用户观测期间购卡gmv 
    ,ride_order_should_pay_amt_t -- 干预组用户观测期间骑行应付gmv 
    ,gmv_t -- 干预组用户观测期间总gmv 
    ,buy_ride_card_cnt_t -- 干预组用户观测期间买卡数量 
    ,use_card_ride_ratio_t -- 干预组用户观测期间用骑行卡订单占比 
    ,order_cnt_t -- 干预组用户观测期间订单数量 
    ,coupon_cnt_t -- 干预组用户观测期间核销订单数量 
    ,order_cnt_15d_t -- 干预组用户观测开始前15天订单 
    ,order_cnt_30d_t -- 干预组用户观测开始前30天订单 
    ,order_cnt_60d_t -- 干预组用户观测开始前60天订单 
    ,order_cnt_90d_t -- 干预组用户观测开始前90天订单 
    ,gmv_amt_15d_t -- 干预组用户观测开始前15天gmv 
    ,gmv_amt_30d_t -- 干预组用户观测开始前30天gmv 
    ,gmv_amt_90d_t -- 干预组用户观测开始前90天gmv 
    ,is_has_ride_card_t -- 干预组用户观测日期当天是否有卡 
    
    ,user_id_c -- 干预组用户id
    ,user_flag_c -- 干预组用户层次
    ,city_comp_type_c -- 干预组用户所在城市竞争类型,独占1，竞争0
    ,event_day_c -- 干预组用户开始观测日期
    ,score_c -- 干预组用户倾向分
    ,buy_ride_card_amt_c -- 干预组用户观测期间购卡gmv
    ,ride_order_should_pay_amt_c -- 干预组用户观测期间骑行应付gmv
    ,gmv_c -- 干预组用户观测期间总gmv
    ,buy_ride_card_cnt_c -- 干预组用户观测期间买卡数量
    ,use_card_ride_ratio_c -- 干预组用户观测期间用骑行卡订单占比
    ,order_cnt_c --
    ,coupon_cnt_c
    ,order_cnt_15d_c
    ,order_cnt_30d_c
    ,order_cnt_60d_c
    ,order_cnt_90d_c
    ,gmv_amt_15d_c
    ,gmv_amt_30d_c
    ,gmv_amt_90d_c
    ,is_has_ride_card_c        
     ,'-1' json
from (
        select 
        a.*
        ,b.user_id_t  user_id_c 
        ,b.user_flag_t user_flag_c
        ,b.city_comp_type_t city_comp_type_c
        ,b.event_day_t event_day_c
        ,b.score_t score_c
        ,b.buy_ride_card_amt_t buy_ride_card_amt_c  
        ,b.ride_order_should_pay_amt_t  ride_order_should_pay_amt_c 
        ,b.gmv_t   gmv_c
        ,b.buy_ride_card_cnt_t  buy_ride_card_cnt_c 
        ,b.use_card_ride_ratio_t   use_card_ride_ratio_c
        ,b.order_cnt_t  order_cnt_c
        ,b.coupon_cnt_t  coupon_cnt_c
        ,b.order_cnt_15d_t order_cnt_15d_c 
        ,b.order_cnt_30d_t  order_cnt_30d_c
        ,b.order_cnt_60d_t  order_cnt_60d_c
        ,b.order_cnt_90d_t   order_cnt_90d_c
        ,b.gmv_amt_15d_t   gmv_amt_15d_c
        ,b.gmv_amt_30d_t   gmv_amt_30d_c
        ,b.gmv_amt_90d_t  gmv_amt_90d_c
        ,b.is_has_ride_card_t  is_has_ride_card_c
        ,row_number() over (partition by a.user_id_t order by abs(a.gmv_amt_30d_t-b.gmv_amt_30d_t) asc) as rn
        from 
              user_t a left join user_c b  on a.user_flag_t = b.user_flag_t and a.city_comp_type_t = b.city_comp_type_t 
                             and b.order_cnt_15d_t BETWEEN a.order_cnt_30d_t-2 and a.order_cnt_15d_t+2
                             and b.order_cnt_30d_t BETWEEN a.order_cnt_30d_t-4 and a.order_cnt_30d_t+4
                             and b.order_cnt_60d_t BETWEEN a.order_cnt_60d_t-6 and a.order_cnt_60d_t+6
                             and b.order_cnt_90d_t BETWEEN a.order_cnt_90d_t-8 and a.order_cnt_30d_t+8
                             
                             and b.gmv_amt_30d_t BETWEEN a.gmv_amt_30d_t-300 and a.gmv_amt_30d_t+300
) 
where rn=1         
"""

### 参考
- https://causalml.readthedocs.io/en/latest/examples.html#propensity-score-matching
- 笔记matching小结
- https://mp.weixin.qq.com/s/pjgYKfFICqzvFwhHwHBNow